https://codeforces.com/gym/384518

https://codeforces.com/gym/384518/problem/A

In [20]:
from collections import defaultdict as dd

def check():
    for key in to_include:
        if to_include[key] > temp[key]:
            return False
    return True

n = int(input())
li = list(map(int, input().split()))

d = dd(int)
for i in range(n):
    d[li[i]] += 1
    
to_include = dd(int)
start = 0
for key in d:
    if d[key] >= 2:
        to_include[key] = d[key] - 1
        start += to_include[key]
        
if start == 0:
    print(0)
else:
    r = -1
    for rc in range(start, n):
        temp = dd(int)
        for i in range(rc):
            temp[li[i]] += 1
            
        if check():
            r = rc
            break
        for cur_end in range(rc, n):
            temp[li[cur_end - rc]] -= 1
            temp[li[cur_end]] += 1
            
            if check():
                r = rc
                break
                
        if r != -1:
            break
            
    print(r)

12
1 3 4 5 6 7 8 9 1 2 2 1
4


In [ ]:
1 3 4 5 6 7 8 9 1 2 2 1

https://codeforces.com/gym/384518/problem/B

In [ ]:
100

123456789
987654321

In [ ]:
2001

   9
   2

  101

In [ ]:
0
못넘김 넘김
0      1~9
0      9~1
1개    9개

1
01     2~9
10     9~2
2개    8개

2
012    3~9
210    9~3
3개    7개

9
0123456789
9876543210
10개       0개

In [51]:
import sys
input = lambda: sys.stdin.readline().rstrip()

def recur(li, i, count):
    global temp
    
    if i == -1:
        temp += count
        return
    
    ncount = count * (li[i] + 1)
    recur(li.copy(), i - 1, ncount)
    
    nli = li.copy()
    ncount = count * (10 - (li[i] + 1))
    
    j = i - 1
    ip = True
    while True:
        if j < 0:
            ip = False
            break
        if 1 <= nli[j]:
            nli[j] -= 1
            break
        nli[j] = 9
        j -= 1
        
    if ip:
        recur(nli, i - 1, ncount)

for _ in range(int(input())):
    li = list(map(int, input()))
    
    a = []
    b = []
    for i in range(len(li)):
        if i % 2 == 0:
            a.append(li[i])
        else:
            b.append(li[i])

    temp = 0
    if a:
        recur(a, len(a) - 1, 1)
    cur = max(temp, 1)

    temp = 0
    if b:
        recur(b, len(b) - 1, 1)
    cur *= max(temp, 1)
    cur -= 2

    print(cur)

2
100
9
12
4


In [ ]:
2, 2, 2, 2
2, 2, 2, 1

1, 0, 0, 2
0, 9, 9, 2

In [47]:
def recur(li, i, count):
    global temp
    
    print(li, i, count)
    
    if i == -1:
        temp += count
        return
    
    ncount = count * (li[i] + 1)
    recur(li.copy(), i - 1, ncount)
    
    nli = li.copy()
    ncount = count * (10 - (li[i] + 1))
    
    j = i - 1
    ip = True
    while True:
        if j < 0:
            ip = False
            break
        if 1 <= nli[j]:
            nli[j] -= 1
            break
        nli[j] = 9
        j -= 1
        
    if ip:
        recur(nli, i - 1, ncount)

In [48]:
temp = 0
recur([1, 0], 1, 1)
temp

[1, 0] 1 1
[1, 0] 0 1
[1, 0] -1 2
[0, 0] 0 9
[0, 0] -1 9


11

https://codeforces.com/gym/384518/problem/C

In [29]:
# https://github.com/tatyam-prime/SortedSet/blob/main/SortedMultiset.py
import math
from bisect import bisect_left, bisect_right, insort
from typing import Generic, Iterable, Iterator, TypeVar, Union, List
T = TypeVar('T')

class SortedMultiset(Generic[T]):
    BUCKET_RATIO = 50
    REBUILD_RATIO = 170

    def _build(self, a=None) -> None:
        "Evenly divide `a` into buckets."
        if a is None: a = list(self)
        size = self.size = len(a)
        bucket_size = int(math.ceil(math.sqrt(size / self.BUCKET_RATIO)))
        self.a = [a[size * i // bucket_size : size * (i + 1) // bucket_size] for i in range(bucket_size)]
    
    def __init__(self, a: Iterable[T] = []) -> None:
        "Make a new SortedMultiset from iterable. / O(N) if sorted / O(N log N)"
        a = list(a)
        if not all(a[i] <= a[i + 1] for i in range(len(a) - 1)):
            a = sorted(a)
        self._build(a)

    def __iter__(self) -> Iterator[T]:
        for i in self.a:
            for j in i: yield j

    def __reversed__(self) -> Iterator[T]:
        for i in reversed(self.a):
            for j in reversed(i): yield j
    
    def __len__(self) -> int:
        return self.size
    
    def __repr__(self) -> str:
        return "SortedMultiset" + str(self.a)
    
    def __str__(self) -> str:
        s = str(list(self))
        return "{" + s[1 : len(s) - 1] + "}"

    def _find_bucket(self, x: T) -> List[T]:
        "Find the bucket which should contain x. self must not be empty."
        for a in self.a:
            if x <= a[-1]: return a
        return a

    def __contains__(self, x: T) -> bool:
        if self.size == 0: return False
        a = self._find_bucket(x)
        i = bisect_left(a, x)
        return i != len(a) and a[i] == x

    def count(self, x: T) -> int:
        "Count the number of x."
        return self.index_right(x) - self.index(x)

    def add(self, x: T) -> None:
        "Add an element. / O(√N)"
        if self.size == 0:
            self.a = [[x]]
            self.size = 1
            return
        a = self._find_bucket(x)
        insort(a, x)
        self.size += 1
        if len(a) > len(self.a) * self.REBUILD_RATIO:
            self._build()

    def discard(self, x: T) -> bool:
        "Remove an element and return True if removed. / O(√N)"
        if self.size == 0: return False
        a = self._find_bucket(x)
        i = bisect_left(a, x)
        if i == len(a) or a[i] != x: return False
        a.pop(i)
        self.size -= 1
        if len(a) == 0: self._build()
        return True

    def lt(self, x: T) -> Union[T, None]:
        "Find the largest element < x, or None if it doesn't exist."
        for a in reversed(self.a):
            if a[0] < x:
                return a[bisect_left(a, x) - 1]

    def le(self, x: T) -> Union[T, None]:
        "Find the largest element <= x, or None if it doesn't exist."
        for a in reversed(self.a):
            if a[0] <= x:
                return a[bisect_right(a, x) - 1]

    def gt(self, x: T) -> Union[T, None]:
        "Find the smallest element > x, or None if it doesn't exist."
        for a in self.a:
            if a[-1] > x:
                return a[bisect_right(a, x)]

    def ge(self, x: T) -> Union[T, None]:
        "Find the smallest element >= x, or None if it doesn't exist."
        for a in self.a:
            if a[-1] >= x:
                return a[bisect_left(a, x)]
    
    def __getitem__(self, x: int) -> T:
        "Return the x-th element, or IndexError if it doesn't exist."
        if x < 0: x += self.size
        if x < 0: raise IndexError
        for a in self.a:
            if x < len(a): return a[x]
            x -= len(a)
        raise IndexError

    def index(self, x: T) -> int:
        "Count the number of elements < x."
        ans = 0
        for a in self.a:
            if a[-1] >= x:
                return ans + bisect_left(a, x)
            ans += len(a)
        return ans

    def index_right(self, x: T) -> int:
        "Count the number of elements <= x."
        ans = 0
        for a in self.a:
            if a[-1] > x:
                return ans + bisect_right(a, x)
            ans += len(a)
        return ans

n, t = map(int, input().split())
li = list(map(int, input().split()))

sli = [0]
cumul = 0
for i in range(n):
    cumul += li[i]
    sli.append(cumul)
    
ms = SortedMultiset()
ms.add(0)
total = 0
for i in range(1, n + 1):
    c = i - ms.index(sli[i] - t + 0.1)
    total += c
    ms.add(sli[i])
    
print(total)

4 -1
-2 1 -2 3
3


https://codeforces.com/gym/384518/problem/D

In [38]:
n = int(input())
a = list(map(int, input().split()))

min_total = float('inf')
for i in range(n):
    total = 0
    cur = 0
    for j in range(i - 1, -1, -1):
        c = cur // a[j] + 1
        total += c
        cur = a[j] * c
        
    cur = 0
    for j in range(i + 1, n):
        c = cur // a[j] + 1
        total += c
        cur = a[j] * c
        
    min_total = min(min_total, total)
    
print(min_total)

8
1 8 2 7 3 6 4 5
16


In [ ]:
8

In [ ]:
5로 최소 10 초과?

In [31]:
10 // 5

2

In [ ]:
5로 최소 9 초과?

In [32]:
9 // 5

1

https://codeforces.com/gym/384518/problem/E

In [9]:
import sys
input = lambda: sys.stdin.readline().rstrip()

cri = 2 * int(1e9)

values = []
cur = 1
d = dict()
seq = 0
while True:
    values.append(cur)
    d[cur] = seq
    seq += 1
    cur *= 2
    if cur > cri:
        break

lv = len(values)

n, q = map(int, input().split())
li = list(map(int, input().split()))
ct = [0] * lv
for v in li:
    ct[d[v]] += 1
    
for _ in range(q):
    b = int(input())
    
    total = 0
    for i in range(lv -1, -1, -1):
        t1 = b // values[i]
        t2 = ct[i]

        c = min(t1, t2)
        total += c
        b -= values[i] * c

    if b != 0:
        print(-1)
    else:
        print(total)

5 4
2 4 8 2 4
3


https://codeforces.com/gym/384518/problem/G

In [ ]:
import sys
input = lambda: sys.stdin.readline().rstrip()

for _ in range(int(input())):
    n, m = map(int, input().split())
    li = [list(map(int, input().split())) for _ in range(n)]
    sli = [sorted(v) for v in li]

    s = -1
    e = -1
    ip = True
    for i in range(n):
        for j in range(m):
            if sli[i][j] != li[i][j]:
                if s == -1:
                    s = j
                elif e == -1:
                    e = j
                else:
                    ip = False
                    break
        if s != -1:
            break

    if ip:
        if s == -1:
            print(1, 1)
        else:
            for i in range(n):
                li[i][s], li[i][e] = li[i][e], li[i][s]
                if li[i] != sli[i]:
                    ip = False
                    break

            if ip:
                print(s + 1, e + 1)
            else:
                print(-1)
    else:
        print(-1)

https://codeforces.com/gym/384518/problem/F

In [48]:
from collections import defaultdict as dd

n = int(input())
li = list(map(int, input().split()))

sli = [0] * (n + 1)
cumul = 0
for i in range(n):
    cumul += li[i]
    sli[i + 1] = cumul
    
d = dd(lambda: -1)
d[0] = 0
total = 0
cur_max = -1
for e in range(1, n + 1):
    Se = sli[e]
    cur_max = max(cur_max, d[Se])
    total += e - cur_max - 1
    d[Se] = e
    
print(total)

1
1
1
